# Metaclasses
> introduction

- toc: false 
- badges: true
- comments: true
- categories: [python]

When we instantiate a class, we create an object with a memory location.

In [72]:
class Foo:
    pass
o = Foo()
print(o)

for example the above object is located at

In [62]:
hex(id(o))

'0x11aec27f0'

similarly, when the above class was created, it also created an object with a memory location. Namely,

In [63]:
hex(id(Foo))

'0x7fb89764d9b8'

What's more, just as we can find out the class of `o`, we can learn what's the class of `Foo`

In [93]:
o.__class__

__main__.Foo

In [94]:
Foo.__class__

type

that's right. The same built in method we use to find the type of objects.

Is `type` also an object? what's its class?

In [96]:
type.__class__

type

`type` itself. So is not turttles all the way down.

When an object is created, its class calls `__new__` which if it hasn´t been explicitly defined, it is inherited from its parent class. And, if the parent class hasn´t been explicitly defined, it is the class `object`. Thus, `o` is created by `object.__new__`

In [79]:
o = object.__new__(Foo)

Just as `o` is created by the method `__new__` of `object`

`Foo` is created by the method `__new__` of `type`

In [97]:
Foo = type.__new__(type, 'Foo', (), {})

or more simply

In [98]:
Foo = type('Foo', (), {})

Since `type` allow us to create classes the same way a class allow us to create objects, `type` is called a metaclass.

In the tuple and dictionary above, we can pass base classes and attributes/methods respectively, that define the class.
For example:

for

In [99]:
class Faa:
    pass

<a id='another_cell'></a>

In [100]:
Foo = type('Foo', (Faa, ), {'attr': 100, 'attr_val': lambda x : x.attr})

is equivalent to

In [101]:
class Foo(Faa):
    attr = 100
    def attr_val(self):
        return self.attr

Now let's suppose we wanted to customize the creation of all classes so that all classes's methods get timed. One option would be to modify `type.__new__` so that all methods in the dictionary argument get decorated with

In [89]:
import types
import time
from functools import wraps
def timeit(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        start = time.time()
        resp = f(*args, **kwargs)
        end = time.time()
        return (resp, end - start)
    return wrapper

However, python doesn´t let us modify `type`'s methods. Instead, it allow us to create a class inheriting from type (a metaclass), create our own `__new__` method, 

In [129]:
class TimeMeta(type):
    def __new__(cls, name, bases, attr):# Replace each function with a decorated version of the function
        for name, value in attr.items():
            if type(value) is types.FunctionType or type(value) is types.MethodType:
                attr[name] = timeit(value)
        # Return a new type called TimeMeta
        return super().__new__(cls, name, bases, attr)

and then define classes using this new created metaclass.

In [130]:
class Animal(metaclass=TimeMeta):
    def talk(self):
        time.sleep(1)
        print("Animal talk")

In [131]:
a = Animal()

In [118]:
a.talk()

Animal talk


(None, 1.0045440196990967)

Also, all classes inheriting from classes thus created, will also have the same metaclass

In [124]:
class Cow(Animal):
    def talk(self):
        time.sleep(1)
        print("Moo")

In [125]:
print(Animal.__class__)

<class '__main__.TimeMeta'>


In [126]:
c = Cow()

In [127]:
c.talk()

Moo


(None, 1.005091667175293)

## References

{{ '[Python: Meta-Programming](https://medium.com/swlh/python-meta-programming-d9e06a4d4240)' | fndetail: 1 }}
{{ '[Great answer from StackOverflow](https://stackoverflow.com/a/6581949)' | fndetail: 2 }}
{{ '[Python Metaclasses](https://realpython.com/python-metaclasses)' | fndetail: 3 }}